In [ ]:
import sys
import os


if os.path.abspath('..') not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

In [ ]:
from tqdm.auto import tqdm
import torch
import gpytorch as gp
import altair as alt
import pandas as pd
import numpy as np

from bi_gp.bilateral_kernel import BilateralKernel

In [ ]:
class ExactGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y):
        likelihood = gp.likelihoods.GaussianLikelihood()
        super().__init__(train_x, train_y, likelihood)
        # self.mean_module = gp.means.ConstantMean()
        # self.covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel(ard_num_dims=train_x.size(-1)))
        self.mean_module = gp.means.ZeroMean()
        self.covar_module = gp.kernels.RBFKernel()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)
      
class BilateralGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y):
        likelihood = gp.likelihoods.GaussianLikelihood()
        super().__init__(train_x, train_y, likelihood)
        # self.mean_module = gp.means.ConstantMean()
        # self.covar_module = gp.kernels.ScaleKernel(BilateralKernel(ard_num_dims=train_x.size(-1)))
        self.mean_module = gp.means.ZeroMean()
        self.covar_module = BilateralKernel()
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
def train(x, y, model, mll, optim):
    model.train()

    optim.zero_grad()

    output = model(x)

    loss = -mll(output, y)

    loss.backward()

    optim.step()

    return { 'train/ll': -loss.detach().item() }


def test(x, y, model, lanc_iter=100, pre_size=0):
    model.eval()

    with torch.no_grad():
#        gp.settings.max_preconditioner_size(pre_size), \
#        gp.settings.max_root_decomposition_size(lanc_iter), \
#        gp.settings.fast_pred_var():
        preds = model(x)

        pred_y = model.likelihood(model(x))
        rmse = (pred_y.mean - y).pow(2).mean(0).sqrt()

    return { 'test/rmse': rmse.item() }


def train_util(model, x, y, lr=0.1, epochs=100):
    mll = gp.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
    optim = torch.optim.Adam(model.parameters(), lr=lr)

    for _ in tqdm(range(epochs)):
        train(x, y, model, mll, optim)

## Toy 4-D GP

In [ ]:
n = 1000
d = 4
x = 2. * torch.rand(n, d) - 1.

with torch.no_grad():
  covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel())
  params = covar_module.state_dict()
  params['raw_outputscale'] = torch.tensor(1.0).log()
  params['base_kernel.raw_lengthscale'] = torch.Tensor([[1.5]]).log()
  covar_module.load_state_dict(params)

  covar = gp.distributions.MultivariateNormal(torch.zeros(n), covariance_matrix=covar_module(x))

rperm = torch.randperm(n)#[:n//2]
train_x = x[rperm]
train_y = (covar.sample() + 0.1 * torch.randn(x.size(0)))[rperm]

#   sample_x = x.squeeze(-1).unsqueeze(0).expand(5, -1).numpy()
#   sample_y = covar.sample(torch.Size([5])).numpy()
#   label = np.repeat(np.array([['a', 'b', 'c', 'd', 'e']]).T, n, axis=1)
#   plot_data = {
#     'x': sample_x.flatten().tolist(),
#     'y': sample_y.flatten().tolist(),
#     'id': label.flatten()
#   }

# alt.Chart(pd.DataFrame(plot_data)).mark_line().encode(x='x', y='y', color='id') +\
# alt.Chart(pd.DataFrame({ 'x': train_x.squeeze(-1).numpy(), 'y': train_y.numpy() })).mark_circle().encode(x='x', y='y')

### Exact GP

In [ ]:
egp = ExactGPModel(train_x, train_y).float()

train_util(egp, train_x, train_y)

In [ ]:
for name, p in egp.named_parameters():
  print(f'{name}.exp', p.data.exp())

### Bilateral GP

In [ ]:
bigp = BilateralGPModel(train_x, train_y).float()

train_util(bigp, train_x, train_y)

In [ ]:
for name, p in bigp.named_parameters():
  print(f'{name}.exp', p.data.exp())